In [1]:
import pandas as pd
import re
import os
import locale
from googletrans import Translator, LANGUAGES
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

In [3]:
#pip install googletrans

In [5]:
data_es = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/data/ES_DATA.csv')
data_uk = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/data/UK_DATA.csv')
data_fr = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/data/FR_DATA.csv')
data_nl = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/data/NL_data_translated.csv.csv')
data = pd.read_csv('/Users/paulinesmirnova/Documents/GitHub/superlexis/merged_cleaned.csv')
data

,headline,news_source,datePublished,region,articleSection,length,byline,text,Unnamed: 8,Unnamed: 9
0,I came to terms with dying in a trench in Afgh...,The Daily Telegraph (London),"June 18, 2018 Monday","Edition 1, National Edition",BUSINESS;NEWS; Pg. 8,1059 words,MATTHEW CAINES,It's apt that Steve McCulley is the first vete...,NaN,NaN
1,Sea salt in my nostrils and history at my side...,The Daily Telegraph (London),"September 7, 2019 Saturday","Edition 1, National Edition",TRAVEL;FEATURES; Pg. 13,1246 words,NaN,The bicycle looked perfect: a pretty shade of ...,NaN,NaN
2,The moment the gun goes in triathlon it starts...,The Daily Telegraph (London),"February 15, 2024 Thursday","Edition 1, National Edition",WOMENS SPORT MONTHLY;SPORT; Pg. 10,972 words,NaN,Lucy Charles-Barclay knows a thing or two abou...,NaN,NaN
3,We experienced haphazard meals and mosquitoes....,The Daily Telegraph (London),"October 14, 2023 Saturday","Edition 1, National Edition",TRAVEL;FEATURES; Pg. 30,1173 words,NaN,"HOT RUNNINGS For me, a non-athlete, running ma...",NaN,NaN
4,20 things to do in North Yorkshire this summer...,The Daily Telegraph (London),"July 31, 2021 Saturday","Edition 1, National Edition","TRAVEL;FEATURES; Pg. 10,11",2215 words,NaN,THE MELLOW MOORS If your only experience of mo...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3550,’Stil asfalt tegen geluidsoverlast’,De Telegraaf,28 november 2019 donderdag,Rotterdam,REGIO; Blz. 14,443 words,Maarten Ritman,VVD boos over plannen auto weren door Maarten ...,NaN,VVD angry about plans to ban cars by Maarten R...
3551,’Verhalen over de geschiedenis’,De Telegraaf,8 mei 2018 dinsdag,Utrecht,REGIO; Blz. 12,264 words,NaN,Nieuwegein wil bewoners horen Van onze Utrecht...,NaN,Nieuwegein wants to hear from residents From o...
3552,"""Waar ligt Doorgaand Verkeer?""",De Telegraaf,"August 14, 2004",NaN,Pg. 16,1391 words,NaN,"ALKMAAR, zaterdag Met het schaamrood op de kak...",Buitenlanders kijken op 'fietsvaarvakantie' in...,"ALKMAAR, Saturday Embarrassed, Italian Fabrizi..."
3553,‘Weg met de rondslingerende deelfietsen in de ...,NRC Handelsblad,03 March 2018,NH zaterdag,Binnenland; Blz. 6,463 words,Lucette Mascini,ABSTRACT Stelling Stadsinitiatief Rotterdam en...,NaN,ABSTRACT Statement City Initiative Rotterdam a...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3555 entries, 0 to 3554
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   headline        3555 non-null   object
 1   news_source     3555 non-null   object
 2   datePublished   3547 non-null   object
 3   region          2333 non-null   object
 4   articleSection  3444 non-null   object
 5   length          3555 non-null   object
 6   byline          2803 non-null   object
 7   text            2895 non-null   object
 8   Unnamed: 8      1119 non-null   object
 9   Unnamed: 9      2571 non-null   object
dtypes: object(10)
memory usage: 277.9+ KB


In [9]:
data = data.drop(columns=['region', 'articleSection', 'byline'])

In [11]:
data.isna().sum()

headline            0
news_source         0
datePublished       8
length              0
text              660
Unnamed: 8       2436
Unnamed: 9        984
dtype: int64

In [13]:
# Convert all text in all columns to lowercase
def to_lowercase(text):
    if isinstance(text, str):
        return text.lower()
    return text

# Apply the lowercase conversion to all columns
data = data.applymap(to_lowercase)

/var/folders/98/bbr4z_5s1yj02ygmn84xmtwc0000gn/T/ipykernel_8393/2740884162.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(to_lowercase)


In [15]:
data

,headline,news_source,datePublished,length,text,Unnamed: 8,Unnamed: 9
0,i came to terms with dying in a trench in afgh...,the daily telegraph (london),"june 18, 2018 monday",1059 words,it's apt that steve mcculley is the first vete...,NaN,NaN
1,sea salt in my nostrils and history at my side...,the daily telegraph (london),"september 7, 2019 saturday",1246 words,the bicycle looked perfect: a pretty shade of ...,NaN,NaN
2,the moment the gun goes in triathlon it starts...,the daily telegraph (london),"february 15, 2024 thursday",972 words,lucy charles-barclay knows a thing or two abou...,NaN,NaN
3,we experienced haphazard meals and mosquitoes....,the daily telegraph (london),"october 14, 2023 saturday",1173 words,"hot runnings for me, a non-athlete, running ma...",NaN,NaN
4,20 things to do in north yorkshire this summer...,the daily telegraph (london),"july 31, 2021 saturday",2215 words,the mellow moors if your only experience of mo...,NaN,NaN
...,...,...,...,...,...,...,...
3550,’stil asfalt tegen geluidsoverlast’,de telegraaf,28 november 2019 donderdag,443 words,vvd boos over plannen auto weren door maarten ...,NaN,vvd angry about plans to ban cars by maarten r...
3551,’verhalen over de geschiedenis’,de telegraaf,8 mei 2018 dinsdag,264 words,nieuwegein wil bewoners horen van onze utrecht...,NaN,nieuwegein wants to hear from residents from o...
3552,"""waar ligt doorgaand verkeer?""",de telegraaf,"august 14, 2004",1391 words,"alkmaar, zaterdag met het schaamrood op de kak...",buitenlanders kijken op 'fietsvaarvakantie' in...,"alkmaar, saturday embarrassed, italian fabrizi..."
3553,‘weg met de rondslingerende deelfietsen in de ...,nrc handelsblad,03 march 2018,463 words,abstract stelling stadsinitiatief rotterdam en...,NaN,abstract statement city initiative rotterdam a...


In [193]:
# Function to extract and format date
def extract_and_format_date(date_str):
    try:
        # Extract the date part using regex to handle different formats
        date_part = date_str.split(' ')[0:3]  # Take only the first three parts
        date_str_cleaned = ' '.join(date_part)
        # Parse the cleaned date string
        date_obj = datetime.strptime(date_str_cleaned, '%B %d, %Y')
        # Format to dd/mm/yyyy
        formatted_date = date_obj.strftime('%d/%m/%Y')
        return formatted_date
    except Exception as e:
        # If parsing fails, return None
        return None

# Apply the function to the column
df['date'] = df['datePublished'].apply(extract_and_format_date)

# Drop the rows where dates could not be parsed
#df = df.dropna(subset=['date'])

/var/folders/98/bbr4z_5s1yj02ygmn84xmtwc0000gn/T/ipykernel_7192/402664260.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['datePublished'].apply(extract_and_format_date)


In [195]:
df

,headline,news_source,datePublished,region,articleSection,length,byline,text,translated,date
12,Música para despertar la plaza,The Guardian,"December 27, 2017 Wednesday",1ª Ed. Madrid Edición,MADRID; Madrid; Pág. 8 [AN] Varios artistas ac...,528 words,ALBA MORALEDA,Un escenario vertical en los balcones de la Ca...,A vertical stage on the balconies of the Casa ...,27/12/2017
19,Carmena hará 33 carriles-bici antes de un año,The Guardian,"December 29, 2015 Tuesday",1ª Ed. Madrid Edición,MADRID; Madrid; Pág. 1,761 words,B. G. GALLO,El Ayuntamiento de Madrid (Ahora Madrid) const...,"Before the end of 2016, the Madrid City Counci...",29/12/2015
22,Daneses por un día,The Guardian,"November 2, 2009 Monday",1ª Ed. Madrid Edición,MADRID; Madrid; Pág. 4,421 words,E. G. SEVILLANO,Gema sabe que es una privilegiada. No todos lo...,Gema knows that she is privileged. Not all Mad...,02/11/2009
33,Caminos desconocidos,The Guardian,"September 3, 2005",Andalucia edition,Deportes; Base; Pg. 42,416 words,PEDRO HORRILLO,En la piel de cualquier ciclista te puedes enc...,In the shoes of any cyclist you can potentiall...,03/09/2005
35,"Libia, inundada y fallida",The Guardian,"September 16, 2023 Saturday",Nacional Edición,OPINIÓN; Base; Pág. 12,590 words,NaN,El ciclón Daniel ha puesto de relieve todas la...,Cyclone Daniel has highlighted all the shortco...,16/09/2023
...,...,...,...,...,...,...,...,...,...,...
972,Tory rebellion in the air over 'farcical' meas...,The Guardian,"November 27, 2020 Friday","Edition 2, National Edition","NEWS; Pg. 6,7",1232 words,Robert Mendick; Danielle Sheridan; Camilla Tom...,TO BORROW from Boris Johnson's beloved cod Lat...,NaN,27/11/2020
976,Turmoil everywhere for investors... and it's n...,The Guardian,"November 20, 2018 Tuesday","Edition 1, National Edition",BUSINESS; Pg. 4,1051 words,Tom Rees,"After a year of unnatural calm on the markets,...",NaN,20/11/2018
977,Unilever hopes to clean up after pounds 1.2bn ...,The Guardian,"September 26, 2009 Saturday",NaN,CITY; Pg. 35,285 words,James Hall,DOVE Soap-to-Lipton Tea manufacturer Unilever ...,NaN,26/09/2009
978,Venture to Madeira's quiet north; Every bit as...,The Guardian,"November 4, 2023 Saturday","Edition 1, National Edition",TRAVEL;FEATURES; Pg. 19,993 words,David Taylor,Walk along the levadas and you'll never be los...,NaN,04/11/2023


# Function to parse dates
def parse_date(date_str):
    try:
        # Try parsing English date
        return datetime.strptime(date_str, '%B %d, %Y %A').strftime('%d/%m/%y')
    except ValueError:
        try:
            # Set locale to Spanish for Spanish dates
            locale.setlocale(locale.LC_TIME, 'es_ES')
            parsed_date = datetime.strptime(date_str, '%d %B %Y %A').strftime('%d/%m/%y')
            locale.setlocale(locale.LC_TIME, 'en_US')  # Reset locale to default
            return parsed_date
        except ValueError:
            return None

# Apply the function to the DataFrame
data_es['date'] = data_es['datePublished'].apply(parse_date)

# Display the DataFrame
print(data_es)